In [3]:
# If you don't wanna change subset ratios through re-partitioning dataset
# Here is the fast script
import numpy as np
import pickle
import json
import hashlib

dataset = "cifar10"

new_testset_ratio = 0.5
new_valset_ratio = 0

partition = pickle.load(open(f"{dataset}/partition.pkl", "rb"))

for i in range(len(partition["data_indices"])):
    indices = np.concatenate(
        [
            partition["data_indices"][i]["train"],
            partition["data_indices"][i]["val"],
            partition["data_indices"][i]["test"],
        ],
        axis=0,
    )
    np.random.shuffle(indices)
    new_testset_size = int(len(indices) * new_testset_ratio)
    new_valset_size = int(len(indices) * new_valset_ratio)

    partition["data_indices"][i]["test"] = indices[:new_testset_size]
    partition["data_indices"][i]["val"] = indices[
        new_testset_size : new_testset_size + new_valset_size
    ]
    partition["data_indices"][i]["train"] = indices[
        new_testset_size + new_valset_size :
    ]

pickle.dump(partition, open(f"{dataset}/partition.pkl", "wb"))

args = json.load(open(f"{dataset}/args.json", "r"))
args["test_ratio"] = new_testset_ratio
args["val_ratio"] = new_valset_ratio
json.dump(args, open(f"{dataset}/args.json", "w"), indent=4)
with open(f"{dataset}/partition_md5.txt", "w") as f:
    f.write(hashlib.md5(json.dumps(args.__dict__).encode()).hexdigest())